*The following study relies on the dataset made by :
F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets:
History and Context. ACM Transactions on Interactive Intelligent
Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages.
DOI=http://dx.doi.org/10.1145/2827872*

In [146]:
import numpy as np
from scipy import sparse

# 1. Presentation of the model
## Question 1.1

###### *Run the function load movielens of movielens utils.py with the correct file name and check that the matrix R has size 943 × 1 682. What is the minidata option doing ?*

We run the function *load_movielens* with the file *u.data*, located in the repertory named "ml-100k".

In [147]:
def load_movielens(filename, minidata=False):
    """
    Cette fonction lit le fichier filename de la base de donnees
    Movielens, par exemple 
    filename = '~/datasets/ml-100k/u.data'
    Elle retourne 
    R : une matrice utilisateur-item contenant les scores
    mask : une matrice valant 1 si il y a un score et 0 sinon
    """

    data = np.loadtxt(filename, dtype=int)

    R = sparse.coo_matrix((data[:, 2], (data[:, 0]-1, data[:, 1]-1)),
                          dtype=float)
    R = R.toarray()  # not optimized for big data

    # code la fonction 1_K
    mask = sparse.coo_matrix((np.ones(data[:, 2].shape),
                              (data[:, 0]-1, data[:, 1]-1)), dtype=bool )
    mask = mask.toarray()  # not optimized for big data

    if minidata is True:
        R = R[0:100, 0:200].copy()
        mask = mask[0:100, 0:200].copy()

    return R, mask

In [148]:
R, mask = load_movielens("ml-100k/u.data")
print("The matrix R is : \n", R)
print("It has size :", R.shape)

The matrix R is : 
 [[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
It has size : (943, 1682)


The matrix R has the required size.

The minidata option limits the R and mask matrices to the 100 first users and 200 first movies : it focuses on less data.
We observe it with the instructions : <br>
$\qquad$   *if minidata is True:* <br>
$\qquad\qquad$        *R = R[0:100, 0:200].copy()* <br>
$\qquad\qquad$        *mask = mask[0:100, 0:200].copy()* <br>
Indeed, if we activate the minidata option with the argument *minidata=True*, we get : 

In [149]:
R_minidata, mask_minidata=load_movielens("ml-100k/u.data", minidata=True)
print("The R matrix size is now :", R_minidata.shape)

The R matrix size is now : (100, 200)


## Question 1.2

###### *How many user and films are there in the database ? What is the total number of grades ?*

As the R matrix is a "user-movie" matrix, there are 943 users and 1682 movies. <br> 
*(We can also find these information in the file READ.me of the directory)*

To get the total number of grades, we can count how many "True" values are in the mask matrix. We can do it with the following instruction :

In [150]:
np.sum(mask)

100000

*(We could also find this number in the READ.me file)*

# 2. Find $P$ when $Q^0$ is fixed
## Question 2.1

###### *Calculate the gradient of function g. We will admit that this gradient is Lipschitz continuous with constant $L_{0}$*

$\qquad g(P)=\frac{1}{2} \| 1_{K}\circ (R-Q^0P)\|^2_{F} + \frac{\rho}{2}\|Q\|^2_{F}+ \frac{\rho}{2}\|P\|^2_{F}$ <br><br>
Let's compute the gradient :

$\qquad\nabla g(P) =\rho P-(Q^0)^T*(\mathbb{1}_{K}\circ(R-Q^0P))$

## Question 2.2

###### *The function objective provided in the file movielens utils.py computes g(P ). Complete this function so that it also computes ∇g(P).*

First we complete the line *grad_P = ...* of the *objective* function with the calculation of $\nabla g$.

In [151]:
def objective(P, Q0, R, mask, rho):
    """
    La fonction objectif du probleme simplifie.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q0 : une matrice de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    """

    tmp = (R - Q0.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))

    grad_P = rho*P - np.transpose(Q0).dot(tmp)

    return val, grad_P

Let's check our calculations :

In [152]:
from scipy.sparse.linalg import svds
from scipy.optimize import check_grad

Q_0, s_values, P_0 = svds(R,k=4) #k=4 : the space F has 4 elements
#We will keep these value to test our algorithms

n_users = R.shape[0] #number of users in the database
n_movies = R.shape[1] #number of movies in the database 
rho=0.3

def func_g(Pvector):
    g, gradg = objective(np.reshape(Pvector, (4, n_movies)), Q_0, R, mask, rho)
    return g

def grad_g_ravel(Pvector):
    g, gradg= objective(np.reshape(Pvector, (4, n_movies)), Q_0, R, mask, rho)
    return np.ravel(gradg)

error = check_grad(func_g, grad_g_ravel, np.ravel(P_0))
relative_error = error/np.linalg.norm(grad_g_ravel(P_0))
print("The error is :", error)
print("The relative error is :", relative_error)

The error is : 1.1689658241361758
The relative error is : 0.0015895163847648654


*The error is a bit big but as the relative error is pretty low $~10^{-3}$ the formula is still admitted.*
*It takes a minute to run*

## Question 2.3

###### *Code a function gradient(g, P0, gamma, epsilon) that minimizes a function g using the gradient method with a constant step size γ, starting from the initial point P0 and with stopping criterion ∥∇g(Pk)∥ ≤ ε.*

Let's code a function that minimizes a function g using the gradient method :

In [153]:
def gradientmethod(func, P0, gamma, epsilon):
    Pk = P0
    g_k, gradg_k = objective(Pk,Q_0,R,mask,rho)
    while (np.linalg.norm(gradg_k)>epsilon): #stop condition
        Pk = Pk-gamma*gradg_k
        g_k, gradg_k = objective(Pk,Q_0,R,mask,rho)
    return Pk, g_k

## Question 2.4

###### *Run the function coded in the previous question in order to minimize g up to the precision ε = 1.*

Let's run our code with $\epsilon = 1$ !
Here, we use the Lipschitz constant for the gamma value : $\gamma = \frac{1}{L_{0}}$

In [154]:
L0 = rho + np.linalg.norm((Q_0.transpose()).dot(Q_0))
P_sol, g_min = gradientmethod(func_g,P_0,1/L0,1)
print("La fonction g minimisée vaut :", g_min)
print("Cette valeur est atteinte au point :", P_sol)

La fonction g minimisée vaut : 369551.5499148193
Cette valeur est atteinte au point : [[ 2.23284225e+00 -4.90664665e-01 -9.98919015e+00 ...  4.74987005e-01
  -3.92212474e-02 -8.67808467e-01]
 [-4.53777371e+00  1.35365298e+01  2.56785789e+00 ... -3.63533351e-01
   3.15394841e-01  1.77455812e-01]
 [-2.02978127e+01 -4.68180885e-01 -1.07452929e+01 ... -3.43502581e-01
   8.21005494e-02  1.54999171e-01]
 [-5.76416726e+01 -2.77454348e+01 -1.98640643e+01 ... -6.09197523e-02
  -6.75813808e-01 -6.32544121e-01]]


## Question 2.5

###### *Add a line search to your gradient method, so that you do not rely on the Lipschitz constant of the gradient any more.*

As the Lipschitz constant isn't always known or doesn't always exist, we have to find another way to find the gamma value. 
For this purpose, we will add a line search to our gradient method :

In [155]:
def gradient_linesearch(func, P0, a, b, epsilon):
    Pk = P0
    beta = 0.5
    gradg_k = objective(Pk,Q0,R,mask,rho)[1]
    while (np.sum(gradg_k)>epsilon):
        gamma = b
        new_P = Pk-gamma*gradg_k #new_P represents P*
        while (func(new_P)>func(Pk)-beta*gamma*(np.linalg.norm(gradg_k)**2)): #we calculate the best value of gamma (= when the inequality is verified, the precision achieved is enough)
            gamma=a*gamma
            new_P = Pk-gamma*gradg_k
        Pk = Pk-gamma*gradg_k
        gradg_k = objective(Pk,Q0,R,mask,rho)[1]
    return Pk, func(Pk)

Here, for each iteration of the first loop (*line4*), we have $\gamma = a^lb$ with :
- $a \in (0,1)$
- $b>0$
- $l$ the number of iterations of the second loop (*line7*)

The second loop stops when the following inequality is verified : <br><center><br>
$g(P^*) ≤ g(P_{k}) - \beta \gamma \|\nabla g(P_{k})\|^2$ </center><br>
with $\; P^* = P_{k} - \gamma \nabla g(P_{k})$

# 3. Resolution of the full problem

We will now work with the $f$ function defined by : <br><center>
$f(P,Q) = \frac{1}{2} \|1_{K} \circ (R − QP)\|^2_{F} + \frac{\rho}{2} \|Q\|^2_{F} + \frac{\rho}{2} \|P\|^2_{F} $
</center>
    
## Question 3.1

###### *By remarking that f is a polynomial of degree 4, show that its gradient is not Lipschitz continuous.*

We know that f is a polynomial of degree 4, thus its gradient is polynomial of degree 3. <br>
Yet, a polynomial can't be Lipschitz continuous if its degree is higher than 1. <br>
It proves that $\nabla f(P,Q)$ is not Lipschitz continuous.

*Explanation* <br>
If $\nabla f(P,Q)$ is Lipschitz continuous, then $\nabla^2 f(P,Q)$ is limited.
Yet, $\nabla f(P,Q)$ is a polynomial of degree 2 : thus, it tends to infinity when its variables tend to infinity. That's why $\nabla^2 f(P,Q)$ is not limited and $\nabla f(P,Q)$ is not Lipschitz continuous

## Question 3.2

###### *Solve Problem (1) by the gradient method with line search until reaching the precision ∥∇f(Pk,Qk)∥ ≤ ε with ε = 100. How do you interpret what the algorithm returns?*

We will now work with a new function which calculates the gradient of the f function according to the matrix P and according to the matrix Q.

In [156]:
def total_objective(P, Q, R, mask, rho):
    """
    La fonction objectif du probleme complet.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q : la variable matricielle de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    grad_Q : le gradient par rapport a Q
    """

    tmp = (R - Q.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q ** 2) + np.sum(P ** 2))

    grad_P = rho*P - np.transpose(Q).dot(tmp)

    grad_Q = rho*Q - tmp.dot(np.transpose(P))

    return val, grad_P, grad_Q


def total_objective_vectorized(PQvec, R, mask, rho):
    """
    Vectorisation of the previous function
    """

    #Reconstruction of P et Q
    n_movies = R.shape[1]
    n_users = R.shape[0]
    F = PQvec.shape[0] // (n_movies + n_users)
    Pvec = PQvec[0:n_movies*F]
    Qvec = PQvec[n_movies*F:]
    P = np.reshape(Pvec, (F, n_movies))
    Q = np.reshape(Qvec, (n_users, F))

    val, grad_P, grad_Q = total_objective(P, Q, R, mask, rho)
    return val, np.concatenate([grad_P.ravel(), grad_Q.ravel()])

In [157]:
def func_f(PQvec):
    return total_objective_vectorized(PQvec, R, mask, rho)[0]

def total_gradient_linesearch(func, PQvec, a, b, epsilon):
    '''The fist part of the algorithm is the same as the previous one gradient_linesearch,
        modified with our new function to calculate the f gradient
        The second part aims to reconstruct the P and Q matrices'''
    gradf_k = total_objective_vectorized(PQvec,R,mask,rho)[1]
    beta=0.5
    while (np.linalg.norm(gradf_k)>epsilon):
        gamma = b
        new_PQvec = PQvec-gamma*gradf_k
        while (func_f(new_PQvec)>func_f(PQvec)-beta*gamma*(np.linalg.norm(gradf_k)**2)):
            gamma=a*gamma
            new_PQvec = PQvec-gamma*gradf_k
        PQvec = PQvec-gamma*gradf_k
        gradf_k = total_objective_vectorized(PQvec,R,mask,rho)[1]
   
    #Reconstruction of P and Q
    n_movies = R.shape[1]
    n_users = R.shape[0]
    F = PQvec.shape[0] // (n_movies + n_users)
    Pvec = PQvec[0:n_movies*F]
    Qvec = PQvec[n_movies*F:]
    P = np.reshape(Pvec, (F, n_movies))
    Q = np.reshape(Qvec, (n_users, F))
    return func_f(PQvec), P, Q

Let's solve the problem reaching the precision $\|\nabla f(P_{k},Q_{k})\|_{F} ≤ \epsilon$ with $\epsilon = 1OO$.

In [158]:
PQvec_0 = np.concatenate([P_0.ravel(), Q_0.ravel()])
f_sol, P_sol, Q_sol = total_gradient_linesearch(func_f,PQvec_0,0.5,10,100)
print("The f function minimized is : ", f_sol)
print("This value is reached with the P matrix :", P_sol)
print("and the Q matrix:", Q_sol)

The f function minimized is :  35902.418468204756
This value is reached with the P matrix : [[ 0.29788874 -0.02930736 -0.80313837 ...  0.08666313  0.11618958
  -0.20905795]
 [-0.12531642  0.66789642  0.35908565 ... -0.26877394  0.01438058
  -0.18951458]
 [-0.57308922  0.01546565 -0.85515517 ... -0.34623039  0.13807309
   0.09761885]
 [-1.92028566 -1.5515266  -1.35239322 ... -0.21170852 -0.73438128
  -0.72712221]]
and the Q matrix: [[-0.67006263 -0.4829061   0.10321342 -2.17098078]
 [-0.1333853  -0.70629995 -0.48334684 -1.76027802]
 [-0.1632304  -0.52540598 -0.47150376 -1.41923831]
 ...
 [-0.24995409 -0.50229173 -0.79537329 -1.90727131]
 [ 0.98464311  0.32004033  0.28051985 -2.26922334]
 [-0.85968469  0.30369437 -0.86102378 -1.98201433]]


Here, what interests us is not the value of the function but rather the P and Q matrices for which this value has been reached.
Indeed, in the long run, the aim is to calculate the R matrix with $R_{u,i} = \sum_{f \in F} Q_{u,f}P_{f,i}$ (with user *u* and movie *i*)

*It takes a few minutes to run the code.*

## Question 3.3

###### *What film would you recommend to user 300 ?*

In [159]:
R_300=Q_sol.dot(P_sol)
R_300_recommendations = list((R_300*np.invert(mask))[300]) #keeps only the movies not yet rated (= not yet seen)
print('The movie I would highly recommend to user 300 is the movie number:', R_300_recommendations.index(max(R_300_recommendations)), 'with a predicted rate of', max(R_300_recommendations),'/5')

print('I woul also recommend :')
for i in range(0, len(R_300_recommendations)):
    if R_300_recommendations[i]>4.2 and R_300_recommendations[i]!=312:
        print(i, 'with a predicted rate of', R_300_recommendations[i], '/5')

The movie I would highly recommend to user 300 is the movie number: 312 with a predicted rate of 4.509350312467269 /5
I woul also recommend :
113 with a predicted rate of 4.330135503678586 /5
168 with a predicted rate of 4.388664305700444 /5
271 with a predicted rate of 4.314356857518943 /5
312 with a predicted rate of 4.509350312467269 /5
407 with a predicted rate of 4.205795085277683 /5
519 with a predicted rate of 4.326337630071839 /5
